In [11]:
import os
import sys
import logging
import pandas as pd
from google.cloud import bigquery
from hashlib import md5
from typing import List
import json


# **** SETUP ****

# global variables for file system/ loading JSON file
JSON_DATA = "../data/standard_format_2024.json"
# project paths  
PROJECT_NAME = "mtg-capstone-414921"
DATASET_NAME = "mtg_cards"

TABLE_METADATA = {
    'standard_format_2024': {
        'table_name': 'standard_format_2024',
        'schema': [
            # schema for the tickets table
            bigquery.SchemaField('id', 'string', mode='REQUIRED'),
            bigquery.SchemaField('name', 'string', mode='REQUIRED'),
            bigquery.SchemaField('set', 'string', mode='REQUIRED'),
            bigquery.SchemaField('rarity', 'string', mode='REQUIRED'),
            bigquery.SchemaField('colors', 'string', mode='NULLABLE'),
            bigquery.SchemaField('mana_cost', 'string', mode='NULLABLE'),
            bigquery.SchemaField('cmc', 'int64', mode='REQUIRED'),
            bigquery.SchemaField('type', 'string', mode='REQUIRED'),
            bigquery.SchemaField('subtypes', 'string', mode='NULLABLE'),
            bigquery.SchemaField('supertypes', 'string', mode='NULLABLE'),
            bigquery.SchemaField('text', 'string', mode='REQUIRED'),
            bigquery.SchemaField('power', 'string', mode='NULLABLE'),
            bigquery.SchemaField('toughness', 'string', mode='NULLABLE'),
            bigquery.SchemaField('artist', 'string', mode='REQUIRED'),
            bigquery.SchemaField('number', 'string', mode='REQUIRED'),
            bigquery.SchemaField('layout', 'string', mode='REQUIRED'),
            bigquery.SchemaField('multiverse_id', 'float64', mode='NULLABLE'),
            bigquery.SchemaField('image_url', 'string', mode='NULLABLE'),
        ],
    },
}

# setup logging and logger
logging.basicConfig(            # setting up the root logger
    format='[%(levelname)-5s][%(asctime)s][%(module)s:%(lineno)04d] : %(message)s',
    level=logging.INFO,
    stream=sys.stdout
)
logger: logging.Logger = logging.getLogger('root')      # alias the root logger as `logger`
logger.setLevel(logging.DEBUG)                          # programmatically reassign the logging level

In [20]:
# Load JSON into pandas dataframe
standard_format_df = pd.read_json('./data/standard_format_2024.json')
standard_format_df = standard_format_df[['id', 'name', 'set', 'rarity', 'colors', 'mana_cost', 'cmc', 'type', 'subtypes', 'supertypes', 'text', 'power', 'toughness', 'artist', 'number', 'layout', 'multiverse_id', 'image_url']]
standard_format_df = standard_format_df.astype({'id': 'str', 'name': 'str', 'set': 'str', 'rarity': 'str', 'colors': 'str', 'mana_cost': 'str', 'type': 'str', 'subtypes': 'str', 'supertypes': 'str', 'text': 'str', 'power': 'str', 'toughness': 'str', 'artist': 'str', 'number': 'str', 'layout': 'str', 'image_url': 'str'})
print(standard_format_df.head())

                                     id  \
0  f10c65cc-1a64-552e-b53c-f825ca89d5eb   
1  cf968d16-027f-5970-a3a3-cb2f49e8dca0   
2  12b1f13f-af09-5197-85f7-104ec020bdd5   
3  13d054c7-4c25-5425-8b39-01fed17ebca1   
4  0fb19178-de6a-5410-9506-4fed53ffbd36   

                                    name  set    rarity colors  mana_cost  \
0            Adeline, Resplendent Cathar  MID      Rare  ['W']  {1}{W}{W}   
1  Ambitious Farmhand // Seasoned Cathar  MID  Uncommon  ['W']     {1}{W}   
2  Ambitious Farmhand // Seasoned Cathar  MID  Uncommon  ['W']       None   
3        Beloved Beggar // Generous Soul  MID  Uncommon  ['W']     {1}{W}   
4        Beloved Beggar // Generous Soul  MID  Uncommon  ['W']       None   

   cmc                               type              subtypes  \
0    3  Legendary Creature — Human Knight   ['Human', 'Knight']   
1    2           Creature — Human Peasant  ['Human', 'Peasant']   
2    2            Creature — Human Knight   ['Human', 'Knight']   
3    2    

In [19]:
standard_table_name = f"{PROJECT_NAME}.{DATASET_NAME}.{TABLE_METADATA['standard_format_2024']['table_name']}"
standard_schema = schema=TABLE_METADATA['standard_format_2024']['schema']
client = bigquery.Client()

def load_table(
    df: pd.DataFrame, 
    client: bigquery.Client, 
    table_name: str, 
    schema: List[bigquery.SchemaField], 
    create_disposition: str = 'CREATE_IF_NEEDED', 
    write_disposition: str = 'WRITE_TRUNCATE'
    ) -> None:
    """load dataframe into bigquery table

    Args:
        df (pd.DataFrame): dataframe to load
        client (bigquery.Client): bigquery client
        table_name (str): full table name including project and dataset id
        schema (List[bigquery.SchemaField]): table schema with data types
        create_disposition (str, optional): create table disposition. Defaults to 'CREATE_IF_NEEDED'.
        write_disposition (str, optional): overwrite table disposition. Defaults to 'WRITE_TRUNCATE'.
    """

    # test table name to be full table name including project and dataset name. It must contain to dots
    assert len(table_name.split('.')) == 3, f"Table name must be a full bigquery table name including project and dataset id: '{table_name}'"
    
    # setup bigquery load job:
    #  create table if needed, replace rows, define the table schema
    job_config = bigquery.LoadJobConfig(
        create_disposition=create_disposition,
        write_disposition=write_disposition,
        schema=schema
    )
    logger.info(f"loading table: '{table_name}'")
    job = client.load_table_from_dataframe(df, destination=table_name, job_config=job_config)
    job.result() 

    # get the resulting table
    table = client.get_table(table_name)
    logger.info(f"loaded {table.num_rows} rows into {table.full_table_id}")

load_table(standard_format_df, client, standard_table_name, standard_schema)

[DEBUG][2024-02-20 15:48:28,628][_default:0255] : Checking /Users/kairo/.creds/mtg-capstone-key.json for explicit credentials as part of auth process...
[DEBUG][2024-02-20 15:48:28,631][_default:0255] : Checking /Users/kairo/.creds/mtg-capstone-key.json for explicit credentials as part of auth process...
[INFO ][2024-02-20 15:48:28,633][3856271840:0034] : loading table: 'mtg-capstone-414921.mtg_cards.standard_format_2024'
[DEBUG][2024-02-20 15:48:28,650][retry:0282] : Converted retries value: 3 -> Retry(total=3, connect=None, read=None, redirect=None, status=None)
[DEBUG][2024-02-20 15:48:28,691][requests:0185] : Making request: POST https://oauth2.googleapis.com/token
[DEBUG][2024-02-20 15:48:28,758][connectionpool:1055] : Starting new HTTPS connection (1): oauth2.googleapis.com:443
[DEBUG][2024-02-20 15:48:28,854][connectionpool:0549] : https://oauth2.googleapis.com:443 "POST /token HTTP/1.1" 200 None
[DEBUG][2024-02-20 15:48:28,908][connectionpool:1055] : Starting new HTTPS connecti